In [8]:
RKP = "DL031"
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
sns.set_theme(style="darkgrid")

Executing op DestroyResourceOp in device /job:localhost/replica:0/task:0/device:CPU:0


In [9]:
# Load datasets and rename columns, load all aqi data but specify metro data name
def loadcsv(city="./data/rkpuram.csv"):
    met = pd.read_csv(city,delimiter=';',skiprows=24)
    aqi = pd.read_csv('./data/station_hour.csv')
    print(aqi.columns)
    met.rename(columns={'# Date': 'Date',}, inplace=True)
    met.rename(columns={'UT time': 'Time',}, inplace=True)
    aqi['Time'] = aqi['Datetime'].str[-8:-3]
    aqi['Date'] = aqi['Datetime'].str[0:10]
    stations = ["DL"+str(x).zfill(3) for x in range(1,39)]
    split_aqi = {}
    for i in range(len(stations)):
        split_aqi[stations[i]] = (aqi[aqi['StationId'] == stations[i]])
    return met,aqi,split_aqi
met,aqi,split_aqi = loadcsv()

C:\Users\confusement\miniconda3\envs\mlc\lib\site-packages\IPython\core\interactiveshell.py:3263: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Index(['StationId', 'Datetime', 'PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3',
       'CO', 'SO2', 'O3', 'Benzene', 'Toluene', 'Xylene', 'AQI', 'AQI_Bucket'],
      dtype='object')


In [27]:
# Pre - processing and loading data
class dataset:
    def __init__(self,met,aqi,split_aqi):
            self.metro_data = met
            self.aqi_data = aqi
            self.split_aqi = split_aqi
    def mergedData(self,station,rlist=['PM2.5','PM10','NO','NO2','NOx','NH3','CO','SO2','O3'],roll=48,shift=48):
        df_aqi = self.getdf(station)
        df = pd.merge(df_aqi, self.metro_data, how='inner', on=['Date', 'Time'])
        print("Merged Dataset Size",len(df))
        
        #Pre Processing merged Data
        df['Year'] = df['Date'].str[0:4]
        df['Month'] = df['Date'].str[5:7].astype(np.float64)
        df['Day'] = df['Date'].str[8:10].astype(np.float64)
        df['Hour'] = df['Time'].str[0:2]
        
        # TRIG TRANSFORMATIONS
        df['windX'] = np.cos(np.deg2rad(df['Wind direction'])) * df['Wind speed']
        df['windY'] = np.sin(np.deg2rad(df['Wind direction'])) * df['Wind speed']
        df['hourX'] = np.cos((df['Hour'].astype(np.float64)-1)*np.pi/24)
        df['hourY'] = np.sin((df['Hour'].astype(np.float64)-1)*np.pi/24)
        df['MonthX'] = np.cos((df['Month'].astype(np.float64)-1)*np.pi/12)
        df['MonthY'] = np.sin((df['Month'].astype(np.float64)-1)*np.pi/12)
        
        import datetime
        df['Date'] = pd.to_datetime(df['Date'])
        df['isWeekend'] =  (df['Date'].dt.dayofweek>=5).astype(int)
        
        df.interpolate(method='linear', limit=5,inplace=True)
        
        # Drop Additional columns
        df.drop('Benzene', axis=1, inplace=True)
        df.drop('Toluene',axis=1, inplace=True)
        df.drop('Xylene', axis=1,inplace=True)
        df.drop('AQI_Bucket',axis=1,inplace=True)
        df.drop('Datetime',axis=1,inplace=True)
        df.drop('StationId',axis=1,inplace=True)
        df.drop('Short-wave irradiation',axis=1,inplace=True)
        df.drop('Date',axis=1,inplace=True)
        df.drop('Time',axis=1,inplace=True)
        
        # Rolling and shifting 
        print("Size before roll",len(df))
        for i in rlist:
            df[i+'_mean'] = df[i].rolling(window=roll, min_periods=int(roll//4)).mean()
            df[i+'_min'] = df[i].rolling(window=roll, min_periods=int(roll/4)).min()
            df[i+'_max'] = df[i].rolling(window=roll, min_periods=int(roll/4)).max()
        for i in rlist:
            df[i] = df[i].shift(-shift)
        df.dropna(inplace=True)
        print("Size after roll",len(df))
        
        return df.copy()
    def getdf(self,station):
        return self.split_aqi[station]
    def plot(self,station):
        df = self.getdf(station)
    def stats(self):
        pass
dat = dataset(met,aqi,split_aqi)
df = dat.mergedData('DL031')

Merged Dataset Size 44035
Size before roll 44035
Size after roll 31275


In [28]:
from sklearn.feature_selection import SelectKBest, chi2, f_classif, f_regression , mutual_info_regression
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
dfTrain = df[:]
X = df[['PM2.5_mean','PM2.5_min','PM2.5_max','Temperature','Relative Humidity','windX','windY','Year','MonthX','MonthY','hourX','hourY','isWeekend']]
y = df['PM2.5']
scaler = StandardScaler()
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.33, random_state=42)
print(scaler.fit(Xtrain))

reg = LinearRegression().fit(scaler.transform(Xtrain), ytrain)
# reg = MLPRegressor(random_state=1, max_iter=300).fit(scaler.transform(Xtrain), ytrain)
# reg = SVR(C=1.0, epsilon=0.2).fit(scaler.transform(Xtrain), ytrain)

testPred = reg.predict(scaler.transform(Xtest))
trainPred = reg.predict(scaler.transform(Xtrain))
mse = np.mean((testPred - np.array(ytest))*(testPred - np.array(ytest)))
print(reg.score(X, y))
print(mean_squared_error(testPred, ytest,squared=False))
print(mean_squared_error(trainPred, ytrain,squared=False))

StandardScaler()
-6325.902946422826
72.40145556005754
73.25310699289916


In [29]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [30]:
tf.debugging.set_log_device_placement(True)

# Place tensors on the CPU
with tf.device('/CPU:0'):
  a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

# Run on the GPU
c = tf.matmul(a, b)
print(c)

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [32]:
from keras.models import Sequential
from keras.layers import Dense

with tf.device('/CPU:0'):
    model = Sequential()
    model.add(Dense(100, input_dim=13, kernel_initializer='normal', activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.summary()
    #Fit
    model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])
    history = model.fit(scaler.transform(Xtrain), ytrain, epochs=1000, batch_size=50,  verbose=1, validation_split=0.2)
    #Print Accuracy
    testPred = model.predict(scaler.transform(Xtest))
    trainPred = model.predict(scaler.transform(Xtrain))
    print(mean_squared_error(testPred, ytest,squared=False))
    print(mean_squared_error(trainPred, ytrain,squared=False))

Executing op RandomStandardNormal in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:CPU:0
Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_53 (Dense)             (None, 100)               1400      
_________________________________________________________________
dense_54 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_55 (Dense)             (None, 1)                 101       
Total params: 11,601
Trainable params: 11,601
Non-trainable params: 0
_________________________________________________________________
Train on 16763 samples, validate on 4191 samples
Epoch 1/1000
Executing op __inference_keras_scratch_graph_1363728 in device /

Epoch 38/1000
16763/16763 [==============================] - 1s 49us/step - loss: 3994.7483 - mse: 3994.7473 - mae: 40.4111 - val_loss: 3916.7457 - val_mse: 3916.7456 - val_mae: 41.1102
Epoch 39/1000
16763/16763 [==============================] - 1s 67us/step - loss: 3977.0177 - mse: 3977.0171 - mae: 40.4057 - val_loss: 3917.4462 - val_mse: 3917.4470 - val_mae: 40.8425
Epoch 40/1000
16763/16763 [==============================] - 1s 61us/step - loss: 3954.3292 - mse: 3954.3293 - mae: 40.2085 - val_loss: 3915.7191 - val_mse: 3915.7202 - val_mae: 41.3890
Epoch 41/1000
16763/16763 [==============================] - 1s 47us/step - loss: 3950.6255 - mse: 3950.6252 - mae: 40.2804 - val_loss: 3886.0537 - val_mse: 3886.0540 - val_mae: 40.9719
Epoch 42/1000
16763/16763 [==============================] - 1s 49us/step - loss: 3917.4142 - mse: 3917.4126 - mae: 40.1087 - val_loss: 3908.0191 - val_mse: 3908.0186 - val_mae: 41.8946
Epoch 43/1000
16763/16763 [==============================] - 1s 44us/s

16763/16763 [==============================] - 1s 45us/step - loss: 3443.9645 - mse: 3443.9631 - mae: 38.1575 - val_loss: 3672.4638 - val_mse: 3672.4631 - val_mae: 40.3993
Epoch 83/1000
16763/16763 [==============================] - 1s 44us/step - loss: 3461.1519 - mse: 3461.1514 - mae: 38.2878 - val_loss: 3633.0713 - val_mse: 3633.0715 - val_mae: 40.0807
Epoch 84/1000
16763/16763 [==============================] - 1s 36us/step - loss: 3428.6363 - mse: 3428.6353 - mae: 38.1144 - val_loss: 3680.0933 - val_mse: 3680.0928 - val_mae: 40.4098
Epoch 85/1000
16763/16763 [==============================] - 1s 36us/step - loss: 3421.3987 - mse: 3421.3999 - mae: 38.1311 - val_loss: 3683.1670 - val_mse: 3683.1658 - val_mae: 40.6148
Epoch 86/1000
16763/16763 [==============================] - 1s 36us/step - loss: 3423.2744 - mse: 3423.2739 - mae: 38.1260 - val_loss: 3617.9405 - val_mse: 3617.9407 - val_mae: 39.6026
Epoch 87/1000
16763/16763 [==============================] - 1s 36us/step - loss: 34

Epoch 126/1000
16763/16763 [==============================] - 1s 45us/step - loss: 3066.4520 - mse: 3066.4536 - mae: 36.3858 - val_loss: 3482.0953 - val_mse: 3482.0955 - val_mae: 39.0467
Epoch 127/1000
16763/16763 [==============================] - 1s 51us/step - loss: 3055.3711 - mse: 3055.3711 - mae: 36.5241 - val_loss: 3460.9782 - val_mse: 3460.9785 - val_mae: 39.5533
Epoch 128/1000
16763/16763 [==============================] - 1s 46us/step - loss: 3042.2402 - mse: 3042.2388 - mae: 36.3615 - val_loss: 3471.4952 - val_mse: 3471.4954 - val_mae: 39.7748
Epoch 129/1000
16763/16763 [==============================] - 1s 46us/step - loss: 3037.4264 - mse: 3037.4272 - mae: 36.5667 - val_loss: 3612.5692 - val_mse: 3612.5688 - val_mae: 40.4091
Epoch 130/1000
16763/16763 [==============================] - 1s 37us/step - loss: 3019.8312 - mse: 3019.8320 - mae: 36.5055 - val_loss: 3466.4333 - val_mse: 3466.4343 - val_mae: 39.5239
Epoch 131/1000
16763/16763 [==============================] - 1s 

Epoch 170/1000
16763/16763 [==============================] - 1s 52us/step - loss: 2778.9956 - mse: 2778.9946 - mae: 35.2734 - val_loss: 3353.5262 - val_mse: 3353.5259 - val_mae: 38.5621
Epoch 171/1000
16763/16763 [==============================] - 1s 49us/step - loss: 2775.0032 - mse: 2775.0029 - mae: 35.3452 - val_loss: 3381.9897 - val_mse: 3381.9905 - val_mae: 39.4346
Epoch 172/1000
16763/16763 [==============================] - 1s 40us/step - loss: 2781.7756 - mse: 2781.7766 - mae: 35.3061 - val_loss: 3380.7024 - val_mse: 3380.7024 - val_mae: 39.0260
Epoch 173/1000
16763/16763 [==============================] - 1s 36us/step - loss: 2758.2327 - mse: 2758.2346 - mae: 35.1211 - val_loss: 3463.3409 - val_mse: 3463.3408 - val_mae: 40.6952
Epoch 174/1000
16763/16763 [==============================] - 1s 37us/step - loss: 2757.0518 - mse: 2757.0522 - mae: 35.1155 - val_loss: 3414.8869 - val_mse: 3414.8870 - val_mae: 39.7253
Epoch 175/1000
16763/16763 [==============================] - 1s 

...

Epoch 785/1000
16763/16763 [==============================] - 1s 36us/step - loss: 1629.2770 - mse: 1629.2771 - mae: 28.5090 - val_loss: 3020.2463 - val_mse: 3020.2471 - val_mae: 37.3709
Epoch 786/1000
16763/16763 [==============================] - 1s 36us/step - loss: 1632.7395 - mse: 1632.7393 - mae: 28.5949 - val_loss: 3052.8765 - val_mse: 3052.8765 - val_mae: 37.5949
Epoch 787/1000
16763/16763 [==============================] - 1s 31us/step - loss: 1624.4184 - mse: 1624.4186 - mae: 28.4769 - val_loss: 3014.9325 - val_mse: 3014.9324 - val_mae: 37.5765
Epoch 788/1000
16763/16763 [==============================] - 0s 29us/step - loss: 1650.9385 - mse: 1650.9392 - mae: 28.8421 - val_loss: 2988.5916 - val_mse: 2988.5918 - val_mae: 37.2358
Epoch 789/1000
16763/16763 [==============================] - 0s 29us/step - loss: 1649.3728 - mse: 1649.3721 - mae: 28.7316 - val_loss: 3028.1631 - val_mse: 3028.1628 - val_mae: 38.0644
Epoch 790/1000
16763/16763 [==============================] - 0s 

Epoch 829/1000
16763/16763 [==============================] - 1s 35us/step - loss: 1597.8915 - mse: 1597.8916 - mae: 28.3639 - val_loss: 3159.0656 - val_mse: 3159.0652 - val_mae: 38.1483
Epoch 830/1000
16763/16763 [==============================] - 1s 36us/step - loss: 1601.1080 - mse: 1601.1083 - mae: 28.4100 - val_loss: 2995.5606 - val_mse: 2995.5608 - val_mae: 37.2332
Epoch 831/1000
16763/16763 [==============================] - 1s 36us/step - loss: 1601.4045 - mse: 1601.4045 - mae: 28.3200 - val_loss: 3045.8057 - val_mse: 3045.8059 - val_mae: 38.1306
Epoch 832/1000
16763/16763 [==============================] - 1s 36us/step - loss: 1594.7942 - mse: 1594.7949 - mae: 28.2974 - val_loss: 2980.0116 - val_mse: 2980.0110 - val_mae: 37.2135
Epoch 833/1000
16763/16763 [==============================] - 1s 30us/step - loss: 1593.7759 - mse: 1593.7761 - mae: 28.2493 - val_loss: 3050.9399 - val_mse: 3050.9414 - val_mae: 37.3258
Epoch 834/1000
16763/16763 [==============================] - 0s 

Epoch 873/1000
16763/16763 [==============================] - 1s 35us/step - loss: 1578.9672 - mse: 1578.9680 - mae: 28.1544 - val_loss: 2967.1104 - val_mse: 2967.1113 - val_mae: 36.8654
Epoch 874/1000
16763/16763 [==============================] - 1s 36us/step - loss: 1564.5645 - mse: 1564.5653 - mae: 28.0603 - val_loss: 2917.2068 - val_mse: 2917.2070 - val_mae: 36.7315
Epoch 875/1000
16763/16763 [==============================] - 1s 36us/step - loss: 1578.7221 - mse: 1578.7216 - mae: 28.2086 - val_loss: 2985.4318 - val_mse: 2985.4316 - val_mae: 37.6054
Epoch 876/1000
16763/16763 [==============================] - 1s 35us/step - loss: 1586.1186 - mse: 1586.1185 - mae: 28.2883 - val_loss: 2972.9400 - val_mse: 2972.9399 - val_mae: 37.0704
Epoch 877/1000
16763/16763 [==============================] - 1s 36us/step - loss: 1580.4252 - mse: 1580.4265 - mae: 28.2300 - val_loss: 2938.8184 - val_mse: 2938.8179 - val_mae: 36.9068
Epoch 878/1000
16763/16763 [==============================] - 1s 

Epoch 917/1000
16763/16763 [==============================] - 1s 32us/step - loss: 1551.4164 - mse: 1551.4169 - mae: 27.9872 - val_loss: 3019.1159 - val_mse: 3019.1160 - val_mae: 37.2965
Epoch 918/1000
16763/16763 [==============================] - 1s 34us/step - loss: 1562.9936 - mse: 1562.9930 - mae: 28.0547 - val_loss: 3000.6093 - val_mse: 3000.6091 - val_mae: 37.8420
Epoch 919/1000
16763/16763 [==============================] - 1s 41us/step - loss: 1546.9161 - mse: 1546.9158 - mae: 27.9441 - val_loss: 2946.4197 - val_mse: 2946.4197 - val_mae: 36.8127
Epoch 920/1000
16763/16763 [==============================] - 1s 51us/step - loss: 1560.4549 - mse: 1560.4558 - mae: 28.0923 - val_loss: 3002.3581 - val_mse: 3002.3582 - val_mae: 37.1194
Epoch 921/1000
16763/16763 [==============================] - 1s 43us/step - loss: 1544.2929 - mse: 1544.2931 - mae: 27.9789 - val_loss: 2939.5659 - val_mse: 2939.5662 - val_mae: 36.8682
Epoch 922/1000
16763/16763 [==============================] - 1s 

Epoch 961/1000
16763/16763 [==============================] - 1s 39us/step - loss: 1511.4240 - mse: 1511.4243 - mae: 27.7086 - val_loss: 2904.5136 - val_mse: 2904.5132 - val_mae: 36.6053
Epoch 962/1000
16763/16763 [==============================] - 1s 42us/step - loss: 1536.1359 - mse: 1536.1367 - mae: 27.9943 - val_loss: 2959.9449 - val_mse: 2959.9448 - val_mae: 36.9211
Epoch 963/1000
16763/16763 [==============================] - 1s 46us/step - loss: 1527.4502 - mse: 1527.4501 - mae: 27.9526 - val_loss: 3036.1429 - val_mse: 3036.1431 - val_mae: 37.5754
Epoch 964/1000
16763/16763 [==============================] - 1s 41us/step - loss: 1524.5711 - mse: 1524.5714 - mae: 27.8143 - val_loss: 3043.3725 - val_mse: 3043.3733 - val_mae: 37.5615
Epoch 965/1000
16763/16763 [==============================] - 1s 43us/step - loss: 1512.4542 - mse: 1512.4535 - mae: 27.8146 - val_loss: 3118.0269 - val_mse: 3118.0273 - val_mae: 38.4840
Epoch 966/1000
16763/16763 [==============================] - 1s 